In [1]:
%%capture
%pip install bs4
%pip install requests_html
%pip install langdetect

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
from requests_html import HTMLSession
from langdetect import detect

In [3]:
#truth values
true = "true" #87 pages of true
mostly_true = "mostly-true" #115 pages
half_true = "half-true" #125 pages
mostly_false = "barely-true" #122 pages
false = "false" #232 pages
pants_on_fire = "pants-fire" #104 pages
truth_values = [true, mostly_true, half_true, mostly_false, false, pants_on_fire]

In [4]:
def getData(url):
    r = s.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    return soup

In [5]:
def getArticle(soup):
    url_list = []
    for i in range(30):
        article = soup.find_all('div', {'class': 'm-statement__quote'})[i]
        url = 'http://www.politifact.com' + str(article.find('a')['href'])
        url_list.append(url)
    return url_list

In [306]:
def getArticleGen(soup):
    url_list = []
    for i in range(20):
        article = soup.find_all('h3', {'class': 'm-teaser__title'})[i]
        url = 'http://www.politifact.com' + str(article.find('a')['href'])
        url_list.append(url)
    return url_list

In [307]:
#multi-page genAI data scraping code
#used for scraping regular politifact news articles
s = HTMLSession()
generativeTask_df = pd.DataFrame(columns=['Statement', 'Tldr_text_statements', 'Text'])
#total of 389 pages of articles dating back to august 2007
for page in range(1, 101):
    if page == 1:
        url = "https://www.politifact.com/article/list/"
    else:
        url = "https://www.politifact.com/article/list/?page=" + str(page)
    for i in getArticleGen(getData(url)):
        truth_soup = getData(i)
    
        #statement scrape
        statement = truth_soup.find('div', {'class': 'm-statement__quote'}).text.strip()
        
        #tldr summary text scrape
        tldr_text_statements = []
        main_content = truth_soup.find('div', {'class': 'global-wrapper'}).find('main', {'class': 'global-content'})
        for i in range(len(main_content.find_all('section', {'class': 't-row'}))):
            if main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('div', {'class': 'short-on-time'}):
                tldr_text = main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('div', {'class': 'short-on-time'})
                if tldr_text.find_all('p'):
                    tldr_text = main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('div', {'class': 'short-on-time'}).find_all('p')
                    for i in tldr_text:
                        cleaned = i.text.replace('\xa0', '')
                        tldr_text_statements.append(cleaned)
                else:
                    tldr_text = tldr_text.find_all('li')
                    for i in tldr_text:
                        cleaned = i.text.replace('\xa0', '')
                        tldr_text_statements.append(cleaned)
            else:
                continue

        
        #full text scrape
        text = ''
        main_content = truth_soup.find('div', {'class': 'global-wrapper'}).find('main', {'class': 'global-content'})
        for i in range(len(main_content.find_all('section', {'class': 't-row'}))):
            if main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('article', {'class': 'm-textblock'}):
                full_text = main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('article', {'class': 'm-textblock'}).find_all('p')
                for i in full_text:
                    text += i.text + ' '
                text = text.replace('\xa0', '')

        new_row = {'Statement': statement, 'Tldr_text_statements': tldr_text_statements, 'Text': text}
        generativeTask_df.loc[len(generativeTask_df)] = new_row

In [ ]:
#multi-page true truth-o-meter scraping code 
s = HTMLSession()
true_df = pd.DataFrame(columns=['Claimer', 'Statement', 'Truth_value', 'Tldr_text_statements', 'Text'])
try:
    value = "true"
    for page in range(1, 88):
        if page == 1:
            url = "https://www.politifact.com/factchecks/list/?ruling=" + value
        else:
            url = "https://www.politifact.com/factchecks/list/?page=" + str(page) + "&ruling=" + value
        for i in getArticle(getData(url)):
            truth_soup = getData(i)
            #claimer scrape
            claimer = truth_soup.find('div', {'class': 'm-statement__meta'}).find('a')['title']

            #statement scrape
            statement = truth_soup.find('div', {'class': 'm-statement__quote'}).text.strip()

            #truth value scrape
            truth_value = truth_soup.find('div', {'class': 'm-statement__meter'}).find('div', {'class': 'c-image'}).find('img')['alt']

            #tldr summary text scrape
            tldr_text_statements = []
            main_content = truth_soup.find('div', {'class': 'global-wrapper'}).find('main', {'class': 'global-content'})
            for i in range(len(main_content.find_all('section', {'class': 't-row'}))):
                if main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('div', {'class': 'short-on-time'}):
                    tldr_text = main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('div', {'class': 'short-on-time'})
                    if tldr_text.find_all('p'):
                        tldr_text = main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('div', {'class': 'short-on-time'}).find_all('p')
                        for i in tldr_text:
                            cleaned = i.text.replace('\xa0', '')
                            tldr_text_statements.append(cleaned)
                    else:
                        tldr_text = tldr_text.find_all('li')
                        for i in tldr_text:
                            cleaned = i.text.replace('\xa0', '')
                            tldr_text_statements.append(cleaned)
                else:
                    continue


            #full text scrape
            text = ''
            main_content = truth_soup.find('div', {'class': 'global-wrapper'}).find('main', {'class': 'global-content'})
            for i in range(len(main_content.find_all('section', {'class': 't-row'}))):
                if main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('article', {'class': 'm-textblock'}):
                    full_text = main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('article', {'class': 'm-textblock'}).find_all('p')
                    for i in full_text:
                        text += i.text + ' '
                    text = text.replace('\xa0', '')

            new_row = {'Claimer': claimer, 'Statement': statement, 'Truth_value': truth_value, 'Tldr_text_statements': tldr_text_statements, 'Text': text}
            true_df.loc[len(true_df)] = new_row
except IndexError:
    pass

In [ ]:
#multi-page mostly true truth-o-meter scraping code 
s = HTMLSession()
mostly_true_df = pd.DataFrame(columns=['Claimer', 'Statement', 'Truth_value', 'Tldr_text_statements', 'Text'])
try:
    value = "mostly-true"
    for page in range(1, 126):
        if page == 1:
            url = "https://www.politifact.com/factchecks/list/?ruling=" + value
        else:
            url = "https://www.politifact.com/factchecks/list/?page=" + str(page) + "&ruling=" + value
        for i in getArticle(getData(url)):
            truth_soup = getData(i)
            #claimer scrape
            claimer = truth_soup.find('div', {'class': 'm-statement__meta'}).find('a')['title']

            #statement scrape
            statement = truth_soup.find('div', {'class': 'm-statement__quote'}).text.strip()

            #truth value scrape
            truth_value = truth_soup.find('div', {'class': 'm-statement__meter'}).find('div', {'class': 'c-image'}).find('img')['alt']

            #tldr summary text scrape
            tldr_text_statements = []
            main_content = truth_soup.find('div', {'class': 'global-wrapper'}).find('main', {'class': 'global-content'})
            for i in range(len(main_content.find_all('section', {'class': 't-row'}))):
                if main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('div', {'class': 'short-on-time'}):
                    tldr_text = main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('div', {'class': 'short-on-time'})
                    if tldr_text.find_all('p'):
                        tldr_text = main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('div', {'class': 'short-on-time'}).find_all('p')
                        for i in tldr_text:
                            cleaned = i.text.replace('\xa0', '')
                            tldr_text_statements.append(cleaned)
                    else:
                        tldr_text = tldr_text.find_all('li')
                        for i in tldr_text:
                            cleaned = i.text.replace('\xa0', '')
                            tldr_text_statements.append(cleaned)
                else:
                    continue


            #full text scrape
            text = ''
            main_content = truth_soup.find('div', {'class': 'global-wrapper'}).find('main', {'class': 'global-content'})
            for i in range(len(main_content.find_all('section', {'class': 't-row'}))):
                if main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('article', {'class': 'm-textblock'}):
                    full_text = main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('article', {'class': 'm-textblock'}).find_all('p')
                    for i in full_text:
                        text += i.text + ' '
                    text = text.replace('\xa0', '')

            new_row = {'Claimer': claimer, 'Statement': statement, 'Truth_value': truth_value, 'Tldr_text_statements': tldr_text_statements, 'Text': text}
            mostly_true_df.loc[len(mostly_true_df)] = new_row
except IndexError:
    pass

In [ ]:
#multi-page barely true truth-o-meter scraping code 
s = HTMLSession()
barely_true_df = pd.DataFrame(columns=['Claimer', 'Statement', 'Truth_value', 'Tldr_text_statements', 'Text'])
try:
    value = "barely-true"
    for page in range(1, 116):
        if page == 1:
            url = "https://www.politifact.com/factchecks/list/?ruling=" + value
        else:
            url = "https://www.politifact.com/factchecks/list/?page=" + str(page) + "&ruling=" + value
        for i in getArticle(getData(url)):
            truth_soup = getData(i)
            #claimer scrape
            claimer = truth_soup.find('div', {'class': 'm-statement__meta'}).find('a')['title']

            #statement scrape
            statement = truth_soup.find('div', {'class': 'm-statement__quote'}).text.strip()

            #truth value scrape
            truth_value = truth_soup.find('div', {'class': 'm-statement__meter'}).find('div', {'class': 'c-image'}).find('img')['alt']

            #tldr summary text scrape
            tldr_text_statements = []
            main_content = truth_soup.find('div', {'class': 'global-wrapper'}).find('main', {'class': 'global-content'})
            for i in range(len(main_content.find_all('section', {'class': 't-row'}))):
                if main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('div', {'class': 'short-on-time'}):
                    tldr_text = main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('div', {'class': 'short-on-time'})
                    if tldr_text.find_all('p'):
                        tldr_text = main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('div', {'class': 'short-on-time'}).find_all('p')
                        for i in tldr_text:
                            cleaned = i.text.replace('\xa0', '')
                            tldr_text_statements.append(cleaned)
                    else:
                        tldr_text = tldr_text.find_all('li')
                        for i in tldr_text:
                            cleaned = i.text.replace('\xa0', '')
                            tldr_text_statements.append(cleaned)
                else:
                    continue


            #full text scrape
            text = ''
            main_content = truth_soup.find('div', {'class': 'global-wrapper'}).find('main', {'class': 'global-content'})
            for i in range(len(main_content.find_all('section', {'class': 't-row'}))):
                if main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('article', {'class': 'm-textblock'}):
                    full_text = main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('article', {'class': 'm-textblock'}).find_all('p')
                    for i in full_text:
                        text += i.text + ' '
                    text = text.replace('\xa0', '')

            new_row = {'Claimer': claimer, 'Statement': statement, 'Truth_value': truth_value, 'Tldr_text_statements': tldr_text_statements, 'Text': text}
            barely_true_df.loc[len(barely_true_df)] = new_row
except IndexError:
    pass

In [ ]:
#multi-page mostly false truth-o-meter scraping code 
s = HTMLSession()
mostly_false_df = pd.DataFrame(columns=['Claimer', 'Statement', 'Truth_value', 'Tldr_text_statements', 'Text'])
try:
    value = "barely-true"
    for page in range(1, 123):
        if page == 1:
            url = "https://www.politifact.com/factchecks/list/?ruling=" + value
        else:
            url = "https://www.politifact.com/factchecks/list/?page=" + str(page) + "&ruling=" + value
        for i in getArticle(getData(url)):
            truth_soup = getData(i)
            #claimer scrape
            claimer = truth_soup.find('div', {'class': 'm-statement__meta'}).find('a')['title']

            #statement scrape
            statement = truth_soup.find('div', {'class': 'm-statement__quote'}).text.strip()

            #truth value scrape
            truth_value = truth_soup.find('div', {'class': 'm-statement__meter'}).find('div', {'class': 'c-image'}).find('img')['alt']

            #tldr summary text scrape
            tldr_text_statements = []
            main_content = truth_soup.find('div', {'class': 'global-wrapper'}).find('main', {'class': 'global-content'})
            for i in range(len(main_content.find_all('section', {'class': 't-row'}))):
                if main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('div', {'class': 'short-on-time'}):
                    tldr_text = main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('div', {'class': 'short-on-time'})
                    if tldr_text.find_all('p'):
                        tldr_text = main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('div', {'class': 'short-on-time'}).find_all('p')
                        for i in tldr_text:
                            cleaned = i.text.replace('\xa0', '')
                            tldr_text_statements.append(cleaned)
                    else:
                        tldr_text = tldr_text.find_all('li')
                        for i in tldr_text:
                            cleaned = i.text.replace('\xa0', '')
                            tldr_text_statements.append(cleaned)
                else:
                    continue


            #full text scrape
            text = ''
            main_content = truth_soup.find('div', {'class': 'global-wrapper'}).find('main', {'class': 'global-content'})
            for i in range(len(main_content.find_all('section', {'class': 't-row'}))):
                if main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('article', {'class': 'm-textblock'}):
                    full_text = main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('article', {'class': 'm-textblock'}).find_all('p')
                    for i in full_text:
                        text += i.text + ' '
                    text = text.replace('\xa0', '')

            new_row = {'Claimer': claimer, 'Statement': statement, 'Truth_value': truth_value, 'Tldr_text_statements': tldr_text_statements, 'Text': text}
            mostly_false_df.loc[len(mostly_false_df)] = new_row
except IndexError:
    pass

In [ ]:
#multi-page false truth-o-meter scraping code 
s = HTMLSession()
false_df = pd.DataFrame(columns=['Claimer', 'Statement', 'Truth_value', 'Tldr_text_statements', 'Text'])
try:
    value = "false"
    for page in range(1, 233):
        if page == 1:
            url = "https://www.politifact.com/factchecks/list/?ruling=" + value
        else:
            url = "https://www.politifact.com/factchecks/list/?page=" + str(page) + "&ruling=" + value
        for i in getArticle(getData(url)):
            truth_soup = getData(i)
            #claimer scrape
            claimer = truth_soup.find('div', {'class': 'm-statement__meta'}).find('a')['title']

            #statement scrape
            statement = truth_soup.find('div', {'class': 'm-statement__quote'}).text.strip()

            #truth value scrape
            truth_value = truth_soup.find('div', {'class': 'm-statement__meter'}).find('div', {'class': 'c-image'}).find('img')['alt']

            #tldr summary text scrape
            tldr_text_statements = []
            main_content = truth_soup.find('div', {'class': 'global-wrapper'}).find('main', {'class': 'global-content'})
            for i in range(len(main_content.find_all('section', {'class': 't-row'}))):
                if main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('div', {'class': 'short-on-time'}):
                    tldr_text = main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('div', {'class': 'short-on-time'})
                    if tldr_text.find_all('p'):
                        tldr_text = main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('div', {'class': 'short-on-time'}).find_all('p')
                        for i in tldr_text:
                            cleaned = i.text.replace('\xa0', '')
                            tldr_text_statements.append(cleaned)
                    else:
                        tldr_text = tldr_text.find_all('li')
                        for i in tldr_text:
                            cleaned = i.text.replace('\xa0', '')
                            tldr_text_statements.append(cleaned)
                else:
                    continue


            #full text scrape
            text = ''
            main_content = truth_soup.find('div', {'class': 'global-wrapper'}).find('main', {'class': 'global-content'})
            for i in range(len(main_content.find_all('section', {'class': 't-row'}))):
                if main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('article', {'class': 'm-textblock'}):
                    full_text = main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('article', {'class': 'm-textblock'}).find_all('p')
                    for i in full_text:
                        text += i.text + ' '
                    text = text.replace('\xa0', '')

            new_row = {'Claimer': claimer, 'Statement': statement, 'Truth_value': truth_value, 'Tldr_text_statements': tldr_text_statements, 'Text': text}
            false_df.loc[len(false_df)] = new_row
except (AttributeError, IndexError) as e:
    pass

In [6]:
#multi-page pants on fire truth-o-meter scraping code 
s = HTMLSession()
pants_fire_df = pd.DataFrame(columns=['Claimer', 'Statement', 'Truth_value', 'Tldr_text_statements', 'Text'])
try:
    value = "pants-fire"
    for page in range(1, 105):
        if page == 1:
            url = "https://www.politifact.com/factchecks/list/?ruling=" + value
        else:
            url = "https://www.politifact.com/factchecks/list/?page=" + str(page) + "&ruling=" + value
        for i in getArticle(getData(url)):
            truth_soup = getData(i)
            #claimer scrape
            claimer = truth_soup.find('div', {'class': 'm-statement__meta'}).find('a')['title']

            #statement scrape
            statement = truth_soup.find('div', {'class': 'm-statement__quote'}).text.strip()

            #truth value scrape
            truth_value = truth_soup.find('div', {'class': 'm-statement__meter'}).find('div', {'class': 'c-image'}).find('img')['alt']

            #tldr summary text scrape
            tldr_text_statements = []
            main_content = truth_soup.find('div', {'class': 'global-wrapper'}).find('main', {'class': 'global-content'})
            for i in range(len(main_content.find_all('section', {'class': 't-row'}))):
                if main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('div', {'class': 'short-on-time'}):
                    tldr_text = main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('div', {'class': 'short-on-time'})
                    if tldr_text.find_all('p'):
                        tldr_text = main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('div', {'class': 'short-on-time'}).find_all('p')
                        for i in tldr_text:
                            cleaned = i.text.replace('\xa0', '')
                            tldr_text_statements.append(cleaned)
                    else:
                        tldr_text = tldr_text.find_all('li')
                        for i in tldr_text:
                            cleaned = i.text.replace('\xa0', '')
                            tldr_text_statements.append(cleaned)
                else:
                    continue


            #full text scrape
            text = ''
            main_content = truth_soup.find('div', {'class': 'global-wrapper'}).find('main', {'class': 'global-content'})
            for i in range(len(main_content.find_all('section', {'class': 't-row'}))):
                if main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('article', {'class': 'm-textblock'}):
                    full_text = main_content.find_all('section', {'class': 't-row'})[i].find('div', {'class': 't-row__center'}).find('article', {'class': 'm-textblock'}).find_all('p')
                    for i in full_text:
                        text += i.text + ' '
                    text = text.replace('\xa0', '')

            new_row = {'Claimer': claimer, 'Statement': statement, 'Truth_value': truth_value, 'Tldr_text_statements': tldr_text_statements, 'Text': text}
            pants_fire_df.loc[len(pants_fire_df)] = new_row
except IndexError as e:
    pass

In [10]:
pants_fire_df

,Claimer,Statement,Truth_value,Tldr_text_statements,Text
0,Facebook posts,“Every President we ever had in the United Sta...,pants-fire,[Claims that politicians are reptilian are par...,Politicians can seem cold-blooded sometimes. B...
1,Viral image,Video shows a President Joe Biden impostor.,pants-fire,[Claims that video footage shows a President J...,A recent video revives the conspiracy theory t...
2,Facebook posts,Este video muestra al Dr. Juan Rivera hablando...,pants-fire,"[El video del Dr. Juan Rivera no es legítimo.,...",Los médicos y profesionales que aparecen en lo...
3,Andy Biggs,The Democrats are pushing “to take in a millio...,pants-fire,[The Democratic comments about refugees so far...,Israeli Defense Forces told people in north Ga...
4,Instagram posts,“There isn't a single video or photo suggestin...,pants-fire,[The Oct. 7 Hamas attack at the Tribe of Nova ...,Editor’s note: This story contains references ...
...,...,...,...,...,...
3085,Bloggers,"Obama says America is great, but let's ""try to...",pants-fire,[],"They have online names like fiberguy, dittohea..."
3086,Hillary Clinton,"Obama ""basically threatened to bomb Pakistan.""",pants-fire,[],"At the Democratic debate in Cleveland, Ohio, o..."
3087,Bloggers,"""Barack Obama loves Che Guevara.""",pants-fire,[],\nBarack Obama has been subjected to chain e-m...
3088,John McCain,"Obama ""suggested bombing Pakistan.""",pants-fire,[],"At a media availability in Columbus, Ohio, on ..."


In [2]:
#get only english news entries from data scrape
def detect_language(text):
    try:
        lang = detect(text)
        return 'English' if lang == 'en' else 'Spanish'
    except:
        return 'Unknown'

In [16]:
pants_fire_df['language'] = pants_fire_df['Statement'].apply(detect_language)
english_df = pants_fire_df[pants_fire_df['language'] == 'English']

,Claimer,Statement,Truth_value,Tldr_text_statements,Text,language
0,Facebook posts,“Every President we ever had in the United Sta...,pants-fire,[Claims that politicians are reptilian are par...,Politicians can seem cold-blooded sometimes. B...,English
1,Viral image,Video shows a President Joe Biden impostor.,pants-fire,[Claims that video footage shows a President J...,A recent video revives the conspiracy theory t...,English
3,Andy Biggs,The Democrats are pushing “to take in a millio...,pants-fire,[The Democratic comments about refugees so far...,Israeli Defense Forces told people in north Ga...,English
4,Instagram posts,“There isn't a single video or photo suggestin...,pants-fire,[The Oct. 7 Hamas attack at the Tribe of Nova ...,Editor’s note: This story contains references ...,English
5,Instagram posts,Less than 48 hours after Israel was attacked “...,pants-fire,[Russia’s invasion of Ukraine has been well do...,"As Russia’s war in Ukraine raged on, a differe...",English
...,...,...,...,...,...,...
3082,Bill Clinton,"""Hillary (Clinton), one time late at night whe...",pants-fire,[],"\n(Published April 11, 2008)\n \nBill Clinton ...",English
3083,Chain email,"""According to The Book of Revelations the anti...",pants-fire,[],\nThe chain e-mail attacks on Sen. Barack Obam...,English
3085,Bloggers,"Obama says America is great, but let's ""try to...",pants-fire,[],"They have online names like fiberguy, dittohea...",English
3086,Hillary Clinton,"Obama ""basically threatened to bomb Pakistan.""",pants-fire,[],"At the Democratic debate in Cleveland, Ohio, o...",English
